In [36]:
#Install Biopython

!pip install biopython


#  Fetch Original Protein Sequence

from Bio import Entrez, SeqIO

Entrez.email = "your_email@example.com"  # Replace with your email

# Original protein (Human hemoglobin)
seq_handle = Entrez.efetch(db="protein", id="NP_000509.1", rettype="fasta", retmode="text")
original_seq = SeqIO.read(seq_handle, "fasta")
seq_handle.close()

print("Original Sequence ID:", original_seq.id)
print("Description:", original_seq.description)
print("Sequence length:", len(original_seq.seq))


#  Run BLAST to Find Top Homologs

from Bio.Blast import NCBIWWW

print("Running BLAST (may take 1-2 minutes)...")
blast_result = NCBIWWW.qblast("blastp", "nr", original_seq.seq)

# Save BLAST results
with open("blast_result.xml", "w") as out_handle:
    out_handle.write(blast_result.read())
blast_result.close()
print("BLAST completed and saved as 'blast_result.xml'")

# Step 3: Parse Top 3 Homologous Sequences

from Bio.Blast import NCBIXML

top_hits_ids = []

with open("blast_result.xml") as result_handle:
    blast_records = NCBIXML.parse(result_handle)
    for blast_record in blast_records:
        for alignment in blast_record.alignments[:3]:  # top 3 hits
            top_hits_ids.append(alignment.accession)
            print(f"Hit: {alignment.title}, Length: {alignment.length}, E-value: {alignment.hsps[0].expect}")

print("Top hit IDs:", top_hits_ids)


#  Fetch Top Homolog Sequences

sequences = [original_seq]  # include original

for seq_id in top_hits_ids:
    handle = Entrez.efetch(db="protein", id=seq_id, rettype="fasta", retmode="text")
    seq_record = SeqIO.read(handle, "fasta")
    sequences.append(seq_record)
    handle.close()

# Save sequences to FASTA
SeqIO.write(sequences, "homologs.fasta", "fasta")
print(f"{len(sequences)} sequences saved to 'homologs.fasta'")


# Perform Pairwise Alignment

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

print("\nPairwise Alignments:\n")

for seq in sequences[1:]:  # skip original
    alignments = pairwise2.align.globalxx(original_seq.seq, seq.seq)
    best_alignment = alignments[0]
    score = best_alignment[2]
    align_length = best_alignment[4]
    matches = sum(res1 == res2 for res1, res2 in zip(best_alignment[0], best_alignment[1]))
    identity = matches / align_length * 100

    print(f"Comparing {original_seq.id} vs {seq.id}")
    print(format_alignment(*best_alignment))
    print(f"Alignment score: {score}, Percent identity: {identity:.2f}%\n")


# Summary Table

import pandas as pd

summary_data = []

for seq in sequences[1:]:
    alignments = pairwise2.align.globalxx(original_seq.seq, seq.seq)
    best_alignment = alignments[0]
    score = best_alignment[2]
    align_length = best_alignment[4]
    matches = sum(res1 == res2 for res1, res2 in zip(best_alignment[0], best_alignment[1]))
    identity = matches / align_length * 100
    summary_data.append([seq.id, len(seq.seq), score, f"{identity:.2f}%"])

df = pd.DataFrame(summary_data, columns=["Sequence ID", "Length", "Alignment Score", "Percent Identity"])
print("\nSummary of Pairwise Alignments:")
print(df)


Original Sequence ID: NP_000509.1
Description: NP_000509.1 hemoglobin subunit beta [Homo sapiens]
Sequence length: 147
Running BLAST (may take 1-2 minutes)...
BLAST completed and saved as 'blast_result.xml'
Hit: gb|AAX37051.1| hemoglobin beta, partial [synthetic construct], Length: 148, E-value: 8.12473e-103
Hit: gb|AAX29557.1| hemoglobin beta, partial [synthetic construct], Length: 148, E-value: 8.77358e-103
Hit: ref|NP_000509.1| hemoglobin subunit beta [Homo sapiens] >ref|XP_003819077.1| hemoglobin subunit beta [Pan paniscus] >ref|XP_508242.1| hemoglobin subunit beta [Pan troglodytes] >sp|P68871.2| RecName: Full=Hemoglobin subunit beta; AltName: Full=Beta-globin; AltName: Full=Hemoglobin beta chain; Contains: RecName: Full=LVV-hemorphin-7; Contains: RecName: Full=Spinorphin [Homo sapiens] >sp|P68872.2| RecName: Full=Hemoglobin subunit beta; AltName: Full=Beta-globin; AltName: Full=Hemoglobin beta chain [Pan paniscus] >sp|P68873.2| RecName: Full=Hemoglobin subunit beta; AltName: Full=